In [1]:
import os, random, uuid, datetime
from decimal import Decimal, ROUND_HALF_UP
from pymongo import MongoClient
from bson.decimal128 import Decimal128

PH_TZ = datetime.timezone(datetime.timedelta(hours=8))

mongo_uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")
client = MongoClient(mongo_uri, tz_aware=True)
db = client[db_name]
col = db["pump_inventories"]

loboc_map = [
  {"dispenser_id": 3, "dispenser_name": "dispenser-three", "pump_id": "5", "product": "regular", "location": "loboc"},
  {"dispenser_id": 3, "dispenser_name": "dispenser-three", "pump_id": "6", "product": "diesel",  "location": "loboc"},
  {"dispenser_id": 4, "dispenser_name": "dispenser-four",  "pump_id": "7", "product": "regular", "location": "loboc"},
  {"dispenser_id": 4, "dispenser_name": "dispenser-four",  "pump_id": "8", "product": "diesel",  "location": "loboc"},
]

prices = {"regular": Decimal("62.50"), "diesel": Decimal("58.00")}
unit = "liter"
calibration_fixed = Decimal("30")

def ph_start_to_utc(d):
    dt_ph = datetime.datetime(d.year, d.month, d.day, 0, 0, 0, tzinfo=PH_TZ)
    return dt_ph.astimezone(datetime.timezone.utc)

def as_dec128(x):
    return Decimal128(Decimal(x).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP))

end_ph = datetime.date(2025, 9, 28)
start_ph = end_ph - datetime.timedelta(days=29)
days = [start_ph + datetime.timedelta(days=i) for i in range(30)]

seed = int(f"{start_ph.year}{start_ph.month:02d}{start_ph.day:02d}")
random.seed(seed)

start_utc = ph_start_to_utc(start_ph)
end_utc_excl = ph_start_to_utc(end_ph + datetime.timedelta(days=1))
col.delete_many({"location": "loboc", "date": {"$gte": start_utc, "$lt": end_utc_excl}})

state = {}
for m in loboc_map:
    key = (m["dispenser_id"], m["product"])
    state[key] = {
        "ei_prev": Decimal("5000"),
        "meter_prev": Decimal(str(random.randint(20000, 50000)))
    }

docs = []
for d in days:
    for m in loboc_map:
        key = (m["dispenser_id"], m["product"])
        BI = state[key]["ei_prev"]
        meter_start = state[key]["meter_prev"]

        vol_min = 120 if m["product"] == "regular" else 150
        vol_max = 650 if m["product"] == "regular" else 600
        liters_sold_wish = Decimal(str(random.randint(vol_min, vol_max)))

        PO = Decimal("0")
        cal_actual = min(calibration_fixed, BI)
        max_cash = max(BI - cal_actual - PO, Decimal("0"))
        cash_actual = min(liters_sold_wish, max_cash)

        EI = BI - cal_actual - PO - cash_actual
        meter_end = meter_start + cal_actual + PO + cash_actual

        P = prices[m["product"]]

        doc = {
            "id": str(uuid.uuid4()),
            "created_at": datetime.datetime.now(datetime.timezone.utc),
            "location": m["location"],
            "date": ph_start_to_utc(d),
            "dispenser_name": m["dispenser_name"],
            "pump_id": str(m["pump_id"]),
            "pump_name": m["product"],
            "product": m["product"],
            "unit": unit,
            "price": as_dec128(P),
            "beginning_inventory": as_dec128(BI),
            "calibration": as_dec128(cal_actual),
            "po": as_dec128(PO),
            "cash": as_dec128(cash_actual),
            "ending_inventory": as_dec128(EI),
            "starting_liter_meter": as_dec128(meter_start),
            "ending_liter_meter": as_dec128(meter_end),
        }
        docs.append(doc)

        state[key]["ei_prev"] = EI
        state[key]["meter_prev"] = meter_end

if docs:
    col.insert_many(docs)

len(docs)


120